In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%load_ext line_profiler

import tensorflow as tf
slim = tf.contrib.slim
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()
#config.gpu_options.allow_growth=True
#config.gpu_options.per_process_gpu_memory_fraction=0.45

import sys
# import os

# SLIM_MODELS_PATH = 'slim/'
# TRANSFORMER_MODELS_PATH = 'transformer/'

# ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
# COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
# PASCAL_VOC_DATA = '/gpfs01/bethge/data/PascalVOC'
MASK_RCNN_MODEL_PATH = 'Mask_RCNN/'
SIAMESE_MASK_RCNN_PATH = './'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
# from samples.coco import coco
# from samples.ade20k import ade20k
# from mrcnn import utils
# from mrcnn import model as modellib
from mrcnn import visualize
    
# if SLIM_MODELS_PATH not in sys.path:
#     sys.path.append(SLIM_MODELS_PATH)
# if TRANSFORMER_MODELS_PATH not in sys.path:
#     sys.path.append(TRANSFORMER_MODELS_PATH)
    
import utils as siamese_utils
# import model as siamese_model
# import config as siamese_config
# import siamese_mrcnn_models as model_zoo
    
# import time
import random
import pickle
import numpy as np
# import skimage.io
# import imgaug
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (12.0, 6.0)

# # from spatial_transformer import transformer

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:98% !important; }</style>"))

# # Root directory of the project
# ROOT_DIR = os.getcwd()

# # Directory to save logs and trained model
# MODEL_DIR = os.path.join(ROOT_DIR, "final_experiments/logs")

# # Local path to trained weights file
# COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

In [ ]:
with open('good-examples.pkl', 'rb') as f:
    targets, images, rois, masks, class_ids, scores, category_names = pickle.load(f)

In [ ]:
category_names

In [ ]:
def display_siamese_instances3(target_list, image_list, boxes_list, masks_list, class_ids_list,
                      scores_list=None, class_names="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None,
                      target_shift=10):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """

    if type(target_list) == list:
        M = int(np.sqrt(len(target_list)))
        if len(target_list) - M**2 > 1e-3:
            M = M + 1
    else:
        M = 1
        target_list = [target_list]
        image_list = [image_list]
        boxes_list = [boxes_list]
        masks_list = [masks_list]
        class_ids_list = [class_ids_list]
        if scores_list is not None:
            scores_list = [scores_list]
    N = M
    
    M = 2
    N = 4

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        from matplotlib.gridspec import GridSpec
        # Use GridSpec to show target smaller than image
        fig = plt.figure(figsize=figsize)
        gs = GridSpec(M, N, hspace=0.02, wspace=0.02, left=0, right=1, bottom=0, top=1)
        auto_show = True

    index = 0
    for m1 in range(M):
        for m2 in range(N):
            ax = plt.subplot(gs[m1, m2])

            if index >= len(target_list):
                continue

            target = target_list[index]
            image = image_list[index]
            boxes = boxes_list[index]
            masks = masks_list[index]
            class_ids = class_ids_list[index]
            class_name = class_names[index]
            scores = scores_list[index]

            # Number of instances
            N = boxes.shape[0]
            if not N:
                print("\n*** No instances to display *** \n")
            else:
                assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

            # Generate random colors
            colors = visualize.random_colors(N)

            # Show area outside image boundaries.
            height, width = image.shape[:2]
            ax.set_ylim(height, 0)
            ax.set_xlim(0, width)
            ax.axis('off')
            #ax.set_title(class_name)
            
            masked_image = image.astype(np.uint32).copy()
            for i in range(N):
                color = colors[i]

                # Bounding box
                if not np.any(boxes[i]):
                    # Skip this instance. Has no bbox. Likely lost in image cropping.
                    continue
                y1, x1, y2, x2 = boxes[i]
                if show_bbox:
                    p = visualize.patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                        alpha=0.7, linestyle="dashed",
                                        edgecolor=color, facecolor='none')
                    ax.add_patch(p)

                # Label
                if not captions:
                    class_id = class_ids[i]
                    score = scores[i] if scores is not None else None
                    x = random.randint(x1, (x1 + x2) // 2)
                    caption = "{:.3f}".format(score) if score else 'no score'
                else:
                    caption = captions[i]
                ax.text(x1, y1 + 8, caption,
                        color='w', size=11, backgroundcolor="none")

                # Mask
                mask = masks[:, :, i]
                if show_mask:
                    masked_image = visualize.apply_mask(masked_image, mask, color)

                # Mask Polygon
                # Pad to ensure proper polygons for masks that touch image edges.
                padded_mask = np.zeros(
                    (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
                padded_mask[1:-1, 1:-1] = mask
                contours = visualize.find_contours(padded_mask, 0.5)
                for verts in contours:
                    # Subtract the padding and flip (y, x) to (x, y)
                    verts = np.fliplr(verts) - 1
                    p = visualize.Polygon(verts, facecolor="none", edgecolor=color)
                    ax.add_patch(p)
            ax.imshow(masked_image.astype(np.uint8))

            target_height, target_width = target.shape[:2]
            min_tar_size = 45
            if target_height < target_width:
                target_width = min_tar_size/target_height * target_width
                target_height = min_tar_size
            else:
                target_height = min_tar_size/target_width * target_height
                target_width = min_tar_size
            ax.imshow(target, extent=[target_shift, target_shift + target_width * 2, height - target_shift, height - target_shift - target_height * 2], zorder=9)
            rect = visualize.patches.Rectangle((target_shift, height - target_shift), target_width * 2, -target_height * 2, linewidth=2, edgecolor='red', facecolor='none', zorder=10)
            ax.add_patch(rect)
            #ax.text(5,30,class_name, color='white', fontsize=18)
            index = index + 1

    if auto_show:
        plt.show()
        
    return

In [ ]:
display_siamese_instances3(targets, images, rois, masks, class_ids, scores, category_names, figsize=(16, 16 /3),
                                         target_shift=-5)

In [ ]:
def save_siamese_instances3(target_list, image_list, boxes_list, masks_list, class_ids_list,
                      scores_list=None, class_names="",
                      figsize=(16, 16),
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None,
                      target_shift=10):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """

    if type(target_list) != list:
        target_list = [target_list]
        image_list = [image_list]
        boxes_list = [boxes_list]
        masks_list = [masks_list]
        class_ids_list = [class_ids_list]
        if scores_list is not None:
            scores_list = [scores_list]

    # If no axis is passed, create one and automatically call show()
    for index in range(len(image_list)):
        fig = plt.figure(figsize=figsize)
        ax = plt.subplot(gs[m1, m2])

        target = target_list[index]
        image = image_list[index]
        boxes = boxes_list[index]
        masks = masks_list[index]
        class_ids = class_ids_list[index]
        class_name = class_names[index]
        scores = scores_list[index]

        # Number of instances
        N = boxes.shape[0]
        if not N:
            print("\n*** No instances to display *** \n")
        else:
            assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

        # Generate random colors
        colors = visualize.random_colors(N)

        # Show area outside image boundaries.
        height, width = image.shape[:2]
        ax.set_ylim(height, 0)
        ax.set_xlim(0, width)
        ax.axis('off')
        #ax.set_title(class_name)

        masked_image = image.astype(np.uint32).copy()
        for i in range(N):
            color = colors[i]

            # Bounding box
            if not np.any(boxes[i]):
                # Skip this instance. Has no bbox. Likely lost in image cropping.
                continue
            y1, x1, y2, x2 = boxes[i]
            if show_bbox:
                p = visualize.patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                    alpha=0.7, linestyle="dashed",
                                    edgecolor=color, facecolor='none')
                ax.add_patch(p)

            # Label
            if not captions:
                class_id = class_ids[i]
                score = scores[i] if scores is not None else None
                x = random.randint(x1, (x1 + x2) // 2)
                caption = "{:.3f}".format(score) if score else 'no score'
            else:
                caption = captions[i]
            ax.text(x1, y1 + 8, caption,
                    color='w', size=11, backgroundcolor="none")

            # Mask
            mask = masks[:, :, i]
            if show_mask:
                masked_image = visualize.apply_mask(masked_image, mask, color)

            # Mask Polygon
            # Pad to ensure proper polygons for masks that touch image edges.
            padded_mask = np.zeros(
                (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
            padded_mask[1:-1, 1:-1] = mask
            contours = visualize.find_contours(padded_mask, 0.5)
            for verts in contours:
                # Subtract the padding and flip (y, x) to (x, y)
                verts = np.fliplr(verts) - 1
                p = visualize.Polygon(verts, facecolor="none", edgecolor=color)
                ax.add_patch(p)
        ax.imshow(masked_image.astype(np.uint8))

        target_height, target_width = target.shape[:2]
        min_tar_size = 45
        if target_height < target_width:
            target_width = min_tar_size/target_height * target_width
            target_height = min_tar_size
        else:
            target_height = min_tar_size/target_width * target_height
            target_width = min_tar_size
        ax.imshow(target, extent=[target_shift, target_shift + target_width * 2, height - target_shift, height - target_shift - target_height * 2], zorder=9)
        rect = visualize.patches.Rectangle((target_shift, height - target_shift), target_width * 2, -target_height * 2, linewidth=2, edgecolor='red', facecolor='none', zorder=10)
        ax.add_patch(rect)
        #ax.text(5,30,class_name, color='white', fontsize=18)
        index = index + 1

    if auto_show:
        plt.show()
        
    return

In [ ]:
with open('false-positives.pkl', 'rb') as f:
    targets, images, rois, masks, class_ids, scores, category_names = pickle.load(f)

In [ ]:
siamese_utils.display_siamese_instances2(targets, images, rois, masks, class_ids, scores, category_names, figsize=(16, 16 * 2 / 3),
                                         target_shift=-5)